In [1]:
import tweepy
from twarc import Twarc2, expansions
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
from nltk.stem.porter import PorterStemmer as PS
import time
import schedule
import datetime
import gc

In [ ]:
#lemmatization
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence

In [ ]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')
doc = nlp('Her bags are better than his.')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
#lemmatization
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def lemma(txt):
    s = ''
    doc = nlp(txt)
    
    for sent in doc.sentences:
        for word in sent.words:
            s += word.lemma + ' '
    
    return s

In [ ]:
df['exTweet'] = df['Tweet'].apply(lemma)
df.to_csv("test.csv")
df.head(10)

In [4]:
df = pd.read_csv('./tweet-of-btc/assembling-log.csv')
df.sum()

収集時刻         2021-09-04 19:42:05.9046782021-09-05 01:57:17....
区間(start)    2021-01-01T00:00:00Z2021-02-01T00:00:00Z2021-0...
区間(end)      2021-02-01T00:00:00Z2021-03-01T00:00:00Z2021-0...
収集件数                                                  11269778
dtype: object

In [36]:
df1 = pd.read_csv('./tweet-of-btc/2021-01_tlist.csv',dtype={'Favorite':'int32','RT':'int32',})
df2 = pd.read_csv('./tweet-of-btc/2021-01_ulist.csv', usecols = ['Following','Followers','author_id'],dtype={'Following':'int32','Followers':'int32',})

In [37]:
print(df1.dtypes)
print(df1.memory_usage(index=False))
print(df2.dtypes)
print(df2.memory_usage(index=False))

Tweet         object
Favorite       int32
RT             int32
created_at    object
author_id      int64
dtype: object
Tweet         12570992
Favorite       6285496
RT             6285496
created_at    12570992
author_id     12570992
dtype: int64
Following    int32
Followers    int32
author_id    int64
dtype: object
Following     5247724
Followers     5247724
author_id    10495448
dtype: int64


In [ ]:
df3 = pd.merge(df1 ,df2, on='author_id')
print("merge完了")
del df1,df2
print("del完了")
df3.drop(index = 'author_id',inplace =True)
print("drop完了")
df3.sort_values(by = 'created_at', ascending = True, inplace = True) 
print("sort完了")


In [ ]:
df3.to_csv('./tweet-of-btc/2021-01_alist.csv', index=False)

In [26]:
print(fav)
print(rt)
print(follow)
print(follower)

[65762, 551838, 884991, 97577, 502693, 101034]
[18354, 822642, 111523, 81375, 76418, 27159]
[4194587, 4194558, 4194444, 4194369, 4194157, 4194078]
[61327137, 61328365, 59693310, 54564083, 59718323, 59725152]


In [5]:
gc.collect()

90

In [4]:
df = pd.read_csv('./tweet-of-btc/2021-01_tlist.csv', usecols = ['Tweet','created_at','author_id'])
df.tail(10)
df.to_csv('./2021-01_test.csv', index=False)